![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_1 Humidity sensor

This short notebook introduces how we access a Phidget device from a Jupyter notebook.

It is important to run this notebook in the Google Chrome web browser, on a desktop or laptop computer. Unfortunately, other browsers (FireFox, Safari, Edge) will not work as they cannot access the USB port that connects to the Phidget device.

### Setting up the Phidget device

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a humidty sensor attached. If it does not, find the sensor and attach it.
    - The humidty senor is a small black box attached with a thin cable. It is marked with the initials "HUM."
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- As mentioned above, make sure you are running this notebook in the Google Chrome browser. 

### Notes on local and remote computer

It is interesting to notice that we are using two computers here. There is the laptop (or desktop) computer sitting in front of you, which is the **local** computer. The Callysto Hub is actually a piece of software running on a cloud-based network computer located at the Cybera offices in Alberta (usually), and we call this the **remote** computer. 

We want to program in Python on the Callysto Hub, which will be running on the remote computer. However, the Phidget device is connected to the local computer (the one in front of you), so we need a way to communicate with the device using the local computer. 

We can do this using the Chrome browser and adding some code in the JavaScript language. Javascript runs on the local computer, in the browser, and so it is able to access the Phidget device. Unfortunately, this means we need to include some JavaScript in our notebook. 

### A bit about JavaScript

JavaScript is just another programming language. It has its own syntax, and there are books and videos that will teach you about JavaScript. We don't want to teach it here, but will introduce some basic features so you can see how it works in a Jupyter notebook. 

First, we often put JavaScript code in a cell by itself, with a header that starts with "%%js". For instance, we might see this in a cell:
```
%%js
let x = 23;
let y = x + 24;
```
The first line tells Jupyter to treat the rest of the cell as JavaScript. The remaining lines are just code, written in JavaScript.

The Jupyter notebook is a little funny about how it treats JavaScript variables, like x and y in the code above. When we first define a variable, we need to attach it to the "window" object of the notebook, otherwise the notebook loses track of the variable. So we attach it, like this:
```
%%js
window.x = 23;
window.y = window.x + 24;
```
Once the variable has been attached to the window, we can use them directly, like this:
```
%%js
y = 2*x + 3*y;
```
So it is no longer necessary to refer to the variable in the long form, window.x or window.y. 

Now, when we want to write something from JavaScript into the Jupyter notebook's output cell, we use the **element** object and attach some text to it. So, to write "Hello, World" in JavaScript and have it appear on the Jupyter notebook, we write something like this:
```
%%js
element.text("Hello, World.");
```

Finally, JavaScript also uses libraries of code. To use the Phidget library, you will see some code like this:
```
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
```
That is just some gobbly gook that gets us access to the code library we need. It's important, but don't worry about it. We will also attach the libary to the window, as a variable window.phidget22, so we can use it later in the code. 

This is all you need to know about JavaScript for this notebook. 



## Overview of the code.

It's a bit tricky to use the Phidget device, as we have to "tell" the computer explicity that we want to use the Phidget. This means writing code that tell the computer that, first, we want to use the USB port and cable to connect to the device, and then we want to use a specific device in the starter kit, and finally we want to read data from that deive. 

We will do this in a step-by-step manner, so we can understand how the connection process works. At the end, it is always good practice to clean things up by closing all the connections. 

The main steps are thus:

0. Import some Python libraries
1. Import the Phidget library, in Javascript
2. Define and open the USB connection
3. Open the Humidity device
4. Read some values from the Humidity device
5. When we are done, close the device and USB connection

Now let's get into the details

## Step 0.
Let's import a few Python libraries that we need. 

In [1]:
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

## Step 1.

We load in the Phidgets22 library of code in JavaScript. Notice the "%%js" at the first line, which indicates this is JavaScript code. We also attach the library to the window, so we can use it in later cells. See the notes on JavaScript above.

In [2]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

<IPython.core.display.Javascript object>

In [3]:
## We pause for a second here, to allow the library to load
sleep(1)

## Step 2.
We now open a USB connection. Again the "%%js" indicates this is JavaScript code. The code requests the USB port to open and access the Phidget Hub. 

In [4]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

<IPython.core.display.Javascript object>

This should open a window that asks you to select the Phidget Hub, and click "OK." It looks like this:
<img src="images/Pconnect.png" alt="Hey"  width = 300 />

In [5]:
## We rest for a bit while the USB port opens
sleep(1)

### Check:
We can verify that the connection is live, by checking the state of the connection. The following line of code should return the value "true." If it does not, try running the cell above again. 

In [6]:
%%js
element.text(usbconn.connected);

<IPython.core.display.Javascript object>

## Step 3.
We make a request to open the humidity sensor. 

Again, Javascript code. We create a connection to the Humidity sensor, then open the connection. 

In [7]:
%%js
window.humSensor = new phidget22.HumiditySensor();
humSensor.open();

<IPython.core.display.Javascript object>

In [8]:
## We rest for a bit while the device connects.
sleep(1)

### Check:
We can verify that the humidity device is live, by checking the state of the sensor. The following line of code should return the value "true." If it does not, try running the cell above again. 

In [9]:
%%js
element.text("Is the Humidity Sensor attached? " + humSensor.attached);

<IPython.core.display.Javascript object>

## Step 4.
We can now access the humidity sensor directly, to get the reading value. 

You can run this cell several times if you like, to see if the humidity value changes.

In [10]:
%%js
element.text(humSensor.humidity);

<IPython.core.display.Javascript object>

### Check:
We can do a loop, to check read the humidity device several times.

If you like, try breathing on the sensor. You should see the reading values change on the display below, as your breath will change the humidity content of the air. 

In [11]:
for i in range(10):
    display(Javascript("element.text(humSensor.humidity);"))
    sleep(.25)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Step 5. Closing. 

Once we are all done, we should close the connection to the phidget. This will leave the phidget free for another notebook to use it, or another person to use the computer with the phidget. 

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 


In [14]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'Click to disconnect'
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

Press this button when you are done, to disconnect the Phidgets


Button(description='Click to disconnect', style=ButtonStyle())

<IPython.core.display.Javascript object>

### Confirm

You can confirm the phidget is open or closed by running the following cell. 

If it is open (true), try clicking the button above, again. 

In [15]:
%%js
element.text("Is the Phidget attached: " + humSensor.attached);

<IPython.core.display.Javascript object>

## In case of errors

What kind of errors can happen? What should they do about them?

Symptom 1. At Step 2, you keep getting "Connection Errors."
- Problem is likely that you are not using the Chrome browser.
- Solution is to switch to Chrome.
    
Symptom 2. At Step 2, the requestWebUSBDeviceAccess() command fails.
- Check that you see an image like this: <img src="images/Pconnect.png" alt="Hey"  width = 300 />
- If you don't see it, this probably means your Phidget Starter Kit is not connected.
- Solution is to plug in the Phidget to your computer's USB port, then try again.
    
    
Symptom 3. At Step 2, the image above appears, but you do not see the VINT device displayed.
- Problem is likely that the Phidget device is not attached to your computer.
- Solution is to plug in the Phidget to your computer's USB port, then try again.



Symptom 4. At Step 2 or 3, the image above appears and you select the VINT device, but nothing else happens.
- Problem is likely that another piece of software is already connected to the device. For instance, if you have another Jupyter notebook using the device, or are using Phidget's MakeCode software.
- Solution is to stop the other software from running.
- In some cases, you might need to shut down all the notebooks and log out of the Callysto Hub. Then log back in and start anew. 


[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)